In [23]:
## Define the version number
version_number = "05_12_2025"

In [1]:
from owlready2 import *

onto = get_ontology("http://purl.obolibrary.org/obo/mondo.owl").load()
term = onto.search_one(iri="http://purl.obolibrary.org/obo/{}".format('MONDO_0018874'))
descendants = term.descendants()

In [2]:
import numpy
print(numpy.__version__)

2.2.5


In [3]:
print(descendants)

{obo.MONDO_0100391, obo.MONDO_0019457, obo.MONDO_0100410, obo.MONDO_0019456, obo.MONDO_0015164, obo.MONDO_0100396, obo.MONDO_0100416, obo.MONDO_0005223, obo.MONDO_0100382, obo.MONDO_0006237, obo.MONDO_0100401, obo.MONDO_0850450, obo.MONDO_0100421, obo.MONDO_0017858, obo.MONDO_0100377, obo.MONDO_0100387, obo.MONDO_0850271, obo.MONDO_0015667, obo.MONDO_0100406, obo.MONDO_0018434, obo.MONDO_0100373, obo.MONDO_0850269, obo.MONDO_0100392, obo.MONDO_0020316, obo.MONDO_0015165, obo.MONDO_0100397, obo.MONDO_0100417, obo.MONDO_0005224, obo.MONDO_0020320, obo.MONDO_0100383, obo.MONDO_0859598, obo.MONDO_0100402, obo.MONDO_0850451, obo.MONDO_0100412, obo.MONDO_0100422, obo.MONDO_0018872, obo.MONDO_0100378, obo.MONDO_0100388, obo.MONDO_0971091, obo.MONDO_0019460, obo.MONDO_0100407, obo.MONDO_0018435, obo.MONDO_0850492, obo.MONDO_0100374, obo.MONDO_0100393, obo.MONDO_0020317, obo.MONDO_0015608, obo.MONDO_0100398, obo.MONDO_0017894, obo.MONDO_0100418, obo.MONDO_0011118, obo.MONDO_0100384, obo.MONDO_0

In [7]:
print(len(descendants))

108


In [21]:
from owlready2 import get_ontology
import pandas as pd

# Load the MONDO ontology (can take time the first time it's downloaded)
mondo_url = "http://purl.obolibrary.org/obo/mondo.owl"
onto = get_ontology(mondo_url).load()

# Replace with your MONDO ID of interest
input_mondo_id = "MONDO_0018874"  # e.g., acute myeloid leukemia

# Search for the class
input_class = onto.search_one(iri="*"+input_mondo_id)

if input_class:
    print(f"Input MONDO term: {input_class.name} - {input_class.label}")

    # Recursive function to get all descendants
    def get_descendants(cls):
        all_desc = set()
        for subclass in cls.subclasses():
            all_desc.add(subclass)
            all_desc.update(get_descendants(subclass))
        return all_desc

    descendants = get_descendants(input_class)

    # Store in DataFrame
    data = []
    for desc in descendants:
        mondo_id = desc.name
        label = desc.label[0] if desc.label else "No label"
        data.append({"object_identifier": mondo_id, "object_name": label})

    df = pd.DataFrame(data).sort_values(by="object_identifier")

    print(f"\nTotal descendants found: {len(df)}\n")
    print(df.head())  # Show a preview

else:
    print("MONDO class not found.")

Input MONDO term: MONDO_0018874 - ['acute myeloid leukemia']

Total descendants found: 107

   object_identifier                                        object_name
89     MONDO_0000875                     adult acute monocytic leukemia
68     MONDO_0004051                 aleukemic monocytic leukemia cutis
94     MONDO_0004644                        subacute monocytic leukemia
87     MONDO_0004996                   childhood acute myeloid leukemia
7      MONDO_0005223  acute myeloid leukemia with minimal differenti...


In [22]:
## replace the _ to : in the object_identifier results
df['object_identifier'] = df['object_identifier'].str.replace('_', ':', regex=False)
df['object_category'] = 'biolink:Disease'

## add constant columns: subject_identifier, subject_name, subject_category, predicate
df['subject_identifier'] = input_mondo_id.replace("_", ":")
df['subject_name'] = 'acute myeloid leukemia'
df['subject_category'] = 'biolink:Disease'
df['predicate'] = 'biolink:subclass_of'

df['knowledge_level'] = 'knowledge_assertion'
df['knowledge_source'] = 'Mondo Disease Ontology'
df['disease_context_qualifier'] = input_mondo_id.replace("_", ":")

print(df.head(20))

   object_identifier                                        object_name  \
89     MONDO:0000875                     adult acute monocytic leukemia   
68     MONDO:0004051                 aleukemic monocytic leukemia cutis   
94     MONDO:0004644                        subacute monocytic leukemia   
87     MONDO:0004996                   childhood acute myeloid leukemia   
7      MONDO:0005223  acute myeloid leukemia with minimal differenti...   
27     MONDO:0005224     acute myeloblastic leukemia without maturation   
8      MONDO:0006237                               granulocytic sarcoma   
75     MONDO:0006861                                    myeloid sarcoma   
70     MONDO:0007896                           acute monocytic leukemia   
50     MONDO:0011118                    bilineal acute myeloid leukemia   
60     MONDO:0012883                       acute promyelocytic leukemia   
4      MONDO:0015164  acute myeloid leukemia and myelodysplastic syn...   
24     MONDO:0015165  acu

In [24]:
## save the biolink compliant dataframe into a csv file

## Define the output path for node & edge files after formatting
download_path = f'/Users/Weiqi0/ISB_working/Ilya_lab/Translator/AML_KG_deployment/biolink_compliant/AML.MONDO_descendants.biolink.compliant.{version_number}.tsv'

df.to_csv(download_path, index=False, sep='\t')
